In [1]:
import os

In [2]:
six_classes = []
for scan in os.listdir("data"):
    cnt = 0
    if scan != "meta.csv":
        for segment in os.listdir(os.path.join("data", scan, "segmentations")):
            if "heart" in segment or "aorta" in segment:
                cnt += 1
            if cnt > 6:
                raise ValueError(scan)
            if cnt == 6:
                six_classes.append(scan)

In [3]:
len(six_classes)

8428

In [4]:
import random

data = set()
while len(data) < 300:
    data.add(random.choice(six_classes))

In [5]:
data_0 = list(data)[:100]
data_1 = list(data)[100:200]
data_val = list(data)[200:]

In [6]:
assert len(data_0) == 100
assert len(data_1) == 100
assert len(data_val) == 100

In [7]:
import shutil
from typing import List


def filter_data(
    data: List[str], folder_name: str, include_aorta: bool
) -> None:
    for scan in os.listdir("data"):
        if scan in data:
            os.mkdir(os.path.join(folder_name, scan))
            os.mkdir(os.path.join(folder_name, scan, "segmentations"))
            shutil.copy(
                os.path.join("data", scan, "ct.nii.gz"),
                os.path.join(folder_name, scan, "ct.nii.gz"),
            )
            for segment in os.listdir(
                os.path.join("data", scan, "segmentations")
            ):
                if "heart" in segment or (
                    include_aorta and "aorta" in segment
                ):
                    shutil.copy(
                        os.path.join("data", scan, "segmentations", segment),
                        os.path.join(
                            folder_name, scan, "segmentations", segment
                        ),
                    )


shutil.rmtree("baseline_train", ignore_errors=True)
os.mkdir("baseline_train")
filter_data(data_0, "baseline_train", True)
shutil.rmtree("additional_train", ignore_errors=True)
os.mkdir("additional_train")
filter_data(data_1, "additional_train", False)
shutil.rmtree("validation", ignore_errors=True)
os.mkdir("validation")
filter_data(data_val, "validation", True)

In [8]:
!rm -rf small_data
!mkdir small_data
!mv baseline_train small_data
!mv additional_train small_data
!mv validation small_data

In [ ]:
!tar -zcf small_data.tar.gz small_data/*

In [ ]:
!ls -lh small_data.tar.gz